# Title ()

### 1. Setup & Installation

In [ ]:
!git clone https://github.com/NudelMaster/deep_learning_hw.git

%pip install -r requirements.txt

### 2. Data Loading
Loads the data if binary files exist, otherwise automatically downloads and stores in /data directory

In [1]:
from torchvision import datasets

train_dataset = datasets.STL10(root='./data', split='train', download=True)
test_dataset = datasets.STL10(root='./data', split='test', download=True)

### 3. Data Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
labels_map = {
    0 : "Airplane", 1 : "Bird", 2 : "Car", 3 : "Cat", 4 : "Deer",
    5 : "Dog", 6 : "Horse", 7 : "Monkey", 8 : "Ship", 9 : "Truck"
}
# plot 4 different examples from each class
# configure the plot
fig, axes = plt.subplots(nrows = 10, ncols = 4, figsize=(10,20))
# iterate over 10 classes
for i in range(10):
    # Filter data for this class
    class_i_examples = train_dataset.data[train_dataset.labels == i]

    # Pick 4 random indices
    idx = np.random.randint(0, class_i_examples.shape[0], 4)

    for j in range(4):
        img = class_i_examples[idx[j]]
        ax = axes[i, j]

        # Call the fixed function
        ax.imshow(np.transpose(img, (1, 2, 0)))
        ax.set_title(labels_map[i])

        # Remove axis ticks for cleanliness
        ax.axis("off")
plt.tight_layout()
plt.show()

### Visualize Augmentations

In [ ]:
sample_img, _ = train_dataset[0] # Get a PIL image
import torchvision.transforms as transforms
# Define individual transforms to show them one by one
crop_tf = transforms.RandomResizedCrop(64)
flip_tf = transforms.RandomHorizontalFlip(p=1.0) # Force flip for the demo
color_tf = transforms.ColorJitter(brightness=0.5)

# Plotting
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
axes[0].imshow(sample_img); axes[0].set_title("Original")
axes[1].imshow(crop_tf(sample_img)); axes[1].set_title("Random Crop")
axes[2].imshow(flip_tf(sample_img)); axes[2].set_title("Horizontal Flip")
axes[3].imshow(color_tf(sample_img)); axes[3].set_title("Color Jitter")
plt.show()

### 4. Prepare Data and configurations

In [2]:
import torch
import models
import trainer
import utils
import data_setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_subset, val_subset, test_subset = data_setup.get_stl10_splits(train_dataset=train_dataset,
                                                                  test_dataset=test_dataset)


### Test accuracy helper

In [3]:

def test_model_performance(model, test_dataset, device, num_workers=0):
    model.eval()
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=num_workers)
    correct = 0
    total = 0

    print("Evaluating on Test Set...")
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Final Accuracy on the 8000 test images: {100 * correct / total:.2f} %')

### 5. Model implementations

In [ ]:
model_names = ['logreg', 'nn', 'cnn', 'mobilenet']

- ### Logistic Regression

In [4]:
# Define hyperparameter grid for logistic regression model
logreg_grid = {
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size': [64, 128],
    'weight_decay': [0, 1e-3, 1e-4],
    'epochs': [15]
}


# Perform grid search for logistic regression
results, best_logreg_config = \
    trainer.run_grid_search(model_name='logreg', \
    train_set = train_subset,\
    val_set = val_subset, \
    param_grid = logreg_grid, \
    device=device, num_workers=8)



# Print top 10 results
sorted_results = sorted(results, key=lambda x: x['best_val_acc'], reverse=True)
print(f"Top 10 logreg Configurations:")
for i, res in enumerate(sorted_results[:10]):
    print(f"Rank {i+1}: Val Acc: {res['best_val_acc']:.2f}% | Config: {res['params']}")



# Run best logistic regression model
best_logreg_config['epochs'] = 100
best_logreg_config['patience'] = 10
best_logreg_config['use_scheduler'] = True

logistic_regression = models.get_model(model_name='logreg', num_classes=10).to(device)


logreg_history = \
    trainer.train_model(model = logistic_regression, \
    train_set=train_subset, \
    val_set=val_subset, \
    config = best_logreg_config,\
    device=device, num_workers=8)


# Visualize training history
utils.plot_training_history(logreg_history, title="Logistic Regression Training History")


# Test logistic regression model
test_model_performance(logistic_regression, test_subset, device, num_workers=8)

Starting grid search for 'logreg' with 36 candidates...


  0%|          | 0/36 [00:00<?, ?it/s]


Evaluating Config: {'optimizer': 'adam', 'learning_rate': 0.01, 'batch_size': 64, 'weight_decay': 0, 'epochs': 15}
Model logreg built successfully.
Starting training for 15 epochs on cuda...
Epoch 1/15 | Train Loss: 27.1495 | Val Loss: 20.3852 | Val Acc: 23.70%
Epoch 10/15 | Train Loss: 25.0535 | Val Loss: 24.9196 | Val Acc: 17.70%
Epoch 15/15 | Train Loss: 26.0056 | Val Loss: 20.4947 | Val Acc: 21.70%
Best Val Acc: 23.70%

Evaluating Config: {'optimizer': 'adam', 'learning_rate': 0.01, 'batch_size': 64, 'weight_decay': 0.001, 'epochs': 15}
Model logreg built successfully.
Starting training for 15 epochs on cuda...
Epoch 1/15 | Train Loss: 28.5132 | Val Loss: 21.9670 | Val Acc: 21.10%
Epoch 10/15 | Train Loss: 23.9146 | Val Loss: 23.2999 | Val Acc: 20.20%
Epoch 15/15 | Train Loss: 25.4553 | Val Loss: 19.9964 | Val Acc: 21.30%
Best Val Acc: 24.50%

Evaluating Config: {'optimizer': 'adam', 'learning_rate': 0.01, 'batch_size': 64, 'weight_decay': 0.0001, 'epochs': 15}
Model logreg built 

Exception ignored in: <function _releaseLock at 0x7fe9b677ce00>
Traceback (most recent call last):
  File "/home/dsi/yefimnu/miniconda3/envs/deep_learning_hw/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 826321, 826322) exited unexpectedly

- ### NN

In [ ]:
# Define hyperparameter grid for neural network model
nn_grid = {
    'hidden_dim': [512, 1024],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size': [32, 64],
    'weight_decay': [0, 1e-3, 1e-4],
    'epochs': [15]
}


# Perform grid search for neural network
results, best_nn_config = \
    trainer.run_grid_search('nn', \
    train_set = train_subset, \
    val_set = val_subset,\
    param_grid = nn_grid, \
    device=device, num_workers=8)


# Print top 10 results
sorted_results = sorted(results, key=lambda x: x['best_val_acc'], reverse=True)
print(f"Top 10 nn Configurations:")
for i, res in enumerate(sorted_results[:10]):
    print(f"Rank {i+1}: Val Acc: {res['best_val_acc']:.2f}% | Config: {res['params']}")



# Run best neural network model
best_nn_config['epochs'] = 120
best_nn_config['patience'] = 10
best_nn_config['use_scheduler'] = True


nn = models.get_model('nn', num_classes=10, hidden_dim=best_nn_config.get('hidden_dim', 512)).to(device)

nn_model, nn_history = trainer.train_model(
    nn,
    train_set=train_subset,
    val_set=val_subset,
    config=best_nn_config,
    device=device,
    num_workers=8
)


# Visualize training history
utils.plot_training_history(nn_history, title="Neural Network Training History")
# Test neural network model
test_model_performance(nn_model, test_subset, device, num_workers=8)




- ### CNN

In [ ]:
# Define hyperparameter grid for convolutional neural network model
cnn_grid = {
    'base_width': [16, 32],
    'optimizer': ['adam', 'sgd'],
    'batch_size': [32, 64],
    'weight_decay': [1e-3, 1e-4],
    'learning_rate': [1e-2, 1e-3, 5e-4],
    'epochs': [15]
}


# Perform grid search for convolutional neural network
results, best_cnn_config = \
    trainer.run_grid_search('cnn', \
    train_set = train_subset, \
    val_set = val_subset, \
    param_grid = cnn_grid, device=device, \
    num_workers=8)


# Print top 10 results
sorted_results = sorted(results, key=lambda x: x['best_val_acc'], reverse=True)
print(f"Top 10 CNN Configurations:")
for i, res in enumerate(sorted_results[:10]):
    print(f"Rank {i+1}: Val Acc: {res['best_val_acc']:.2f}% | Config: {res['params']}")


# Run best convolutional neural network model
best_cnn_config['epochs'] = 150
best_cnn_config['patience'] = 20
best_cnn_config['use_scheduler'] = True

cnn = models.get_model('cnn', num_classes=10, **best_cnn_config).to(device)

cnn_history = \
    trainer.train_model(cnn, \
    train_set = train_subset, \
    val_set = val_subset, \
    config = best_cnn_config, \
    device=device, num_workers=8)


# Plot training history for the best CNN model
utils.plot_training_history(cnn_history)



# Test model performance
test_model_performance(cnn, test_subset, device, num_workers=8)

 - ### MobileNet as Feature Extraction with Frozen weights

In [ ]:
# Define hyperparameter grid for MobileNet model with frozen backbone
mobilenet_frozen_grid = {
    'freeze_backbone': [True],
    'weight_decay': [1e-3, 1e-4],
    'hidden_dim': [256, 512],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size': [32,64],
    'epochs': [15]
}

# Perform grid search for MobileNet (frozen)
results, best_mobile_net_frozen_config = trainer.run_grid_search('mobilenet', \
    train_set = train_subset, \
    val_set = val_subset, \
    param_grid = mobilenet_frozen_grid,\
    device=device, \
    num_workers=8)


# Print top 10 results
print("Top 10 MobileNet (frozen) Configurations:")
sorted_results = sorted(results, key=lambda x: x['best_val_acc'], reverse=True)
for i, res in enumerate(sorted_results[:10]):
    print(f"Rank {i+1}: Val Acc: {res['best_val_acc']:.2f}% | Config: {res['params']}")

# Run best MobileNet (frozen) model

best_mobile_net_frozen_config['epochs'] = 100
best_mobile_net_frozen_config['patience'] = 20
best_mobile_net_frozen_config['use_scheduler'] = True


mobile_net_frozen = \
    models.get_model('mobilenet', num_classes=10, **best_mobile_net_frozen_config).to(device)


mobile_net_frozen_history = \
    trainer.train_model(mobile_net_frozen, \
    train_set = train_subset, \
    val_set = val_subset, \
    config = best_mobile_net_frozen_config, \
    device=device, num_workers=8)


# Plot training history for the best MobileNet (frozen) model
utils.plot_training_history(mobile_net_frozen_history, title="MobileNet (frozen) Training History")

# Test MobileNet (frozen) model
test_model_performance(mobile_net_frozen, test_subset, device, num_workers=8)



- ### Mobile Net finetuned

In [ ]:
# Define hyperparameter grid for MobileNet model (fine-tuned)
mobilenet_grid = {
    'freeze_backbone': [False],
    'weight_decay': [1e-3, 1e-4],
    'hidden_dim': [256, 512],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size': [32,64],
    'epochs': [15]
}

# Run grid search for MobileNet (fine-tuned)
results, best_mobile_net_config = \
    trainer.run_grid_search('mobilenet', \
    train_set = train_subset, \
    val_set = val_subset, \
    param_grid = mobilenet_grid, \
    device=device, \
    num_workers=8)

# Print top 10 results
print("Top 10 MobileNet (fine-tuned) Configurations:")
sorted_results = sorted(results, key=lambda x: x['best_val_acc'], reverse=True)
for i, res in enumerate(sorted_results[:10]):
    print(f"Rank {i+1}: Val Acc: {res['best_val_acc']:.2f}% | Config: {res['params']}")

# Run best MobileNet (fine-tuned) model

best_mobile_net_config['epochs'] = 100
best_mobile_net_config['patience'] = 20
best_mobile_net_config['use_scheduler'] = True


mobile_net_fine_tuned = models.get_model('mobilenet', num_classes=10, **best_mobile_net_config).to(device)


mobile_net_fine_tuned_history = \
    trainer.train_model(mobile_net_fine_tuned, \
    train_set = train_subset, \
    val_set = val_subset, \
    config = best_mobile_net_config,\
    device=device, num_workers=8)

# Plot training history for the best MobileNet (fine-tuned) model
utils.plot_training_history(mobile_net_fine_tuned_history)

# Test MobileNet (fine-tuned) model
test_model_performance(mobile_net_fine_tuned, test_subset, device, num_workers=8)